In [1]:
import tensorflow as tf

In [2]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6, 7])

In [3]:
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [4]:
for ele in dataset:
    print(ele.numpy())

1
2
3
4
5
6
7


In [5]:
dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4], [5, 6]])

In [6]:
for ele in dataset:
    print(ele.numpy())

[1 2]
[3 4]
[5 6]


In [7]:
dataset_dic = tf.data.Dataset.from_tensor_slices({
    'a': [1, 2, 3, 4],
    'b': [6, 7, 8, 9],
    'c': [12, 13, 14, 15]
})

In [8]:
for ele in dataset_dic:
    print(ele)

{'a': <tf.Tensor: shape=(), dtype=int32, numpy=1>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=6>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=12>}
{'a': <tf.Tensor: shape=(), dtype=int32, numpy=2>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=13>}
{'a': <tf.Tensor: shape=(), dtype=int32, numpy=3>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=14>}
{'a': <tf.Tensor: shape=(), dtype=int32, numpy=4>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=9>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=15>}


In [9]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

In [10]:
for ele in dataset.take(4):
    print(ele.numpy())

1
2
3
4


In [11]:
next(iter(dataset.take(1)))

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [12]:
dataset = dataset.shuffle(7)
# 乱序

In [13]:
for ele in dataset:
    print(ele.numpy())

2
6
3
1
5
4


In [14]:
dataset = dataset.repeat(count=3)
for ele in dataset:
    print(ele.numpy())

5
2
4
6
3
1
4
1
5
2
3
6
4
5
3
6
2
1


In [15]:
dataset = dataset.batch(3)
for ele in dataset:
    print(ele.numpy())

[5 6 3]
[1 4 2]
[3 1 6]
[5 2 4]
[5 4 6]
[2 3 1]


In [16]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6, 7])

In [17]:
dataset = dataset.map(tf.square)
# .map 对数据进行操作
for ele in dataset:
    print(ele.numpy())

1
4
9
16
25
36
49


### tf.data训练fashion_mnist数据

In [18]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [19]:
train_images = train_images/255
test_images = test_images/255

In [20]:
ds_train_img = tf.data.Dataset.from_tensor_slices(train_images)

In [21]:
ds_train_lab = tf.data.Dataset.from_tensor_slices(train_labels)

In [22]:
ds_train_lab

<TensorSliceDataset shapes: (), types: tf.uint8>

In [23]:
ds_train = tf.data.Dataset.zip((ds_train_img, ds_train_lab))

In [24]:
ds_train

<ZipDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

In [25]:
ds_train = ds_train.shuffle(10000).repeat().batch(64)

In [37]:
ds_train

<BatchDataset shapes: ((None, 28, 28), (None,)), types: (tf.float64, tf.uint8)>

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [27]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [28]:
steps_per_epochs = train_images.shape[0]/64

In [29]:
model.fit(ds_train, epochs=5, steps_per_epoch=steps_per_epochs)

Epoch 1/5
938/937 [==============================] - 2s 2ms/step - loss: 0.2979 - accuracy: 0.9170
Epoch 2/5
938/937 [==============================] - 2s 2ms/step - loss: 0.1351 - accuracy: 0.9609
Epoch 3/5
938/937 [==============================] - 2s 2ms/step - loss: 0.0954 - accuracy: 0.9713
Epoch 4/5
938/937 [==============================] - 2s 2ms/step - loss: 0.0721 - accuracy: 0.9786
Epoch 5/5
938/937 [==============================] - 2s 2ms/step - loss: 0.0565 - accuracy: 0.9831


In [30]:
ds_test = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [38]:
ds_test = ds_test.batch(64)

In [33]:
steps_per_epochs = train_images.shape[0]//64

In [39]:
model.fit(ds_train,
          epochs=5,
          steps_per_epoch=steps_per_epochs,
          validation_data=ds_test,
          validation_steps=10000//64)

Epoch 1/5
937/937 [==============================] - 2s 2ms/step - loss: 0.0202 - accuracy: 0.9946 - val_loss: 0.0790 - val_accuracy: 0.9764
Epoch 2/5
937/937 [==============================] - 2s 2ms/step - loss: 0.0176 - accuracy: 0.9950 - val_loss: 0.0773 - val_accuracy: 0.9784
Epoch 3/5
937/937 [==============================] - 2s 2ms/step - loss: 0.0141 - accuracy: 0.9961 - val_loss: 0.0809 - val_accuracy: 0.9777
Epoch 4/5
937/937 [==============================] - 2s 2ms/step - loss: 0.0119 - accuracy: 0.9971 - val_loss: 0.0812 - val_accuracy: 0.9783
Epoch 5/5
937/937 [==============================] - 2s 2ms/step - loss: 0.0103 - accuracy: 0.9975 - val_loss: 0.0880 - val_accuracy: 0.9774
